In [1]:
import os
import sys
import pickle
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms

import models

from PIL import Image
import torchvision.utils as vutils

import random
from tqdm import tqdm

def manual_seed(seed):
    np.random.seed(seed) #1
    random.seed(seed) #2
    torch.manual_seed(seed) #3Q
    torch.cuda.manual_seed(seed) #4.1
    torch.cuda.manual_seed_all(seed) #4.2
    torch.backends.cudnn.benchmark = False #5 
    torch.backends.cudnn.deterministic = True #6
    


In [2]:
manual_seed(42)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

batch_size = 64
val_batch_size = 256

In [4]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Resize((32,32)),
])

train_dataset = torchvision.datasets.FashionMNIST(root='./data',train=True, download=False, transform=train_transforms)
train_loader = DataLoader(train_dataset,batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True,drop_last=True)

val_transforms = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Resize((32,32)),
])

val_dataset = torchvision.datasets.FashionMNIST(root='./data',train=False,download=False ,transform=val_transforms)
val_loader = DataLoader(val_dataset,batch_size=val_batch_size, shuffle=False, num_workers=8, pin_memory=True)

print(len(val_dataset.data), len(val_loader))


10000 40


In [5]:
EPOCHS = 10
LR = 1e-4

In [6]:
model = models.Classifier()
optimizer = torch.optim.Adam(model.parameters(), lr = LR,  weight_decay=0.005)
# scheduler = None
criterion = torch.nn.CrossEntropyLoss()

In [7]:
model.to(device)
best_acc = 0
for epoch in range(EPOCHS):
    running_loss = 0
    running_acc = 0
    model.train()

    for imgs, labels in tqdm(train_loader, leave=False):
        imgs = torch.flatten(imgs,1)
        imgs = imgs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        preds = torch.argmax(outputs, dim=-1)
        running_loss += loss.item()
        running_acc += (preds==labels).sum().item()
    running_acc = running_acc /len(train_dataset)*100
    print(f"{epoch+1} training_loss : {running_loss /len(train_loader):.2f} acc :{running_acc:.2f}%, lr = {optimizer.param_groups[0]['lr']}")

    # val
    model.eval()
    val_loss = 0
    val_acc = 0
    count = 0
    with torch.no_grad():
        for imgs, labels in tqdm(val_loader, leave=False):
            imgs = torch.flatten(imgs,1)
            imgs = imgs.to(device)
            labels = labels.to(device)
            
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            preds = torch.argmax(outputs, dim=-1)

            val_loss += loss.item()
            val_acc += (preds==labels).sum().item()
            count +=256
    val_acc = val_acc/ len(val_dataset)*100
    print(f"\tval_loss : {val_loss /len(val_loader):.2f} acc :{val_acc:.2f}%")
    if best_acc < val_acc:
        best_acc = val_acc
        checkpoint = {
            'model_state_dict' : model.state_dict(),
        }
        torch.save(checkpoint, './check/clacheckpoint.pth')
        print(f"save best acc {best_acc:.2f}")

1 training_loss : 0.77 acc :75.75%, lr = 0.0001


	val_loss : 0.57 acc :80.66%
save best acc 80.66


2 training_loss : 0.52 acc :82.67%, lr = 0.0001


	val_loss : 0.52 acc :82.54%
save best acc 82.54


3 training_loss : 0.49 acc :83.84%, lr = 0.0001


...

	val_loss : 0.46 acc :83.83%


9 training_loss : 0.43 acc :85.49%, lr = 0.0001


	val_loss : 0.45 acc :84.49%
save best acc 84.49


10 training_loss : 0.43 acc :85.66%, lr = 0.0001


	val_loss : 0.45 acc :84.55%
save best acc 84.55
